In [31]:
import pandas as pd
import os
import gzip
import pickle
import openai
import torch.nn.functional as F
import torch
import re
import copy
import tqdm

In [32]:
def get_openai_response(prompt, tokens = 99999, model="gpt-4"):
    # model="gpt-3.5-turbo-16k"
    response = openai.ChatCompletion.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "Always answer with the requested csv file."
            },
            {
                "role": "user",
                "content": prompt
            }
            ],
        temperature=0,
        # max_tokens=tokens,
    )
    text = response.choices[0].message.content


    # print("returning ai text:", text)
    return text

In [33]:
df = pd.read_csv('datasets/extracted/date_name_price_original.csv')

# sort by date
df = df.sort_values(by=['date'])

# add row index
df = df.reset_index(drop=True)
df["row_index"] = df.index

# move row index left
cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

df_price_only = copy.deepcopy(df)
df_name_only = copy.deepcopy(df)
df_date_only = copy.deepcopy(df)

In [34]:
# remove rows where hardware_price contains the string "unknown"
df_price_only = df_price_only[df_price_only['hardware_price'].str.contains("unknown") == False]

df_price_only = df_price_only.drop(columns=['date'])
df_price_only = df_price_only.drop(columns=['hardware_name'])

# save
df_price_only.to_csv('datasets/extracted/price_only.csv', index=False)

In [35]:
# drop the date, hardware_price columns
df_name_only = df_name_only.drop(columns=['date'])
df_name_only = df_name_only.drop(columns=['hardware_price'])

# save
df_name_only.to_csv('datasets/extracted/name_only.csv', index=False)

In [36]:
# drop the hardware_price, hardware_name columns
df_date_only = df_date_only.drop(columns=['hardware_price'])
df_date_only = df_date_only.drop(columns=['hardware_name'])

# save
df_date_only.to_csv('datasets/extracted/date_only.csv', index=False)

# GPT-4 processing (prices)

In [6]:
#read the file back as a string
with open('datasets/extracted/price_only.csv', 'r') as file:
    table_text = file.read()

prompt = f"""
{table_text}


The prices in this csv need to be cleaned.
Rewrite the csv with the following change:
Rewrite the price in a standard format, use this format for all rows: "1234$" or "1234BTC" or "1234€" etc. depending on the currency
If a price is a range, keep only the first price. For example "6$-7$" should be written as "6$"
If a price has no currency then assume it is in dollars if it is above 5, otherwise assume it is in BTC
If a price is not valid, for example "USB bitcoin miners", write "unknown"
If a price is per unit of something then here are some examples of how to handle them:
"1.1 btc/Blade" should be written as "1.1BTC per Blade"
".071 / 13 GH/s" should be written as "0.071BTC per 13 GH/s"
"0.40btc each" should be written as "0.40BTC per unit"
"0.5 BTC shares" should be written as "0.5BTC per share"
etc.

""".strip()



print(prompt)

# response = get_openai_response(prompt) # the response is a csv as a string with columns date,hardware_name,hardware_price,hashrate,efficiency


# # save the response
# with open('datasets/extracted/name_and_price_cleaned.csv', 'w') as file:
#     file.write(response)


# df = pd.read_csv('datasets/extracted/price_only_cleaned.csv')


row_index,hardware_price
5,$70
11,$2.10 per hour
13,$200
17,$220
22,$55
23,$79
33,$185
36,$99
45,$280
46,$126
47,$288
52,$109
58,$150
59,$200-300
85,$800
102,107 BTC
103,$580
104,$89
113,$150
135,26 BTC
142,$75 and $101
148,$80
150,$275
151,"$2,000"
152,$1200
153,$200
158,807.1501152800002 BTC
164,$599
171,0.079 BTC
173,$320
176,$2.03636
177,$2.03636
182,$5000
183,4.5 Ghz
184,0.5 BTC shares
185,60EUR
190,0.082BTC
191,0.5 BTC shares
192,2.3 BTC
193,0.5 BTC shares
196,20.7
197,60EUR
198,0.082BTC
199,$2.04236
200,2.04236 btc
209,1 BTC per share
210,0.65 BTC
213,175 GHs Miner
214,.57 btc per chip
215,4.56 BTC
217,4 Btc
224,1.01618 ea.
226,0.082BTC
227,75$
228,60EUR
230,0.081 BTC
235,$7k
237,0.99 BTC
238,1.01693878 ea.
239,$59/chip
240,0.89BTC
241,1BTC/share
243,1btc= 3 GH/s
244,$0.95 per device
245,BTC1.15
246,BTC1.15
247,0.95 BTC
251,4BTC
253,$65
262,"$27,5/chip"
265,.41 to .55 btc
266,$58
267,$20
271,$58
272,0.35 BTC
273,$45
274,1.475 BTC
275,0.10 btc
276,1.65 btc
277,฿1.65
282,1k
285,$4

# adding efficiency and hashrate

In [7]:
#read the file back as a string
with open('datasets/extracted/name_only.csv', 'r') as file:
    table_text = file.read()
    # table_text = table_text.replace("hardware_name", "").strip()



with open('hardware_index.csv', 'r') as file:
    hardware = file.read()

def split_string(s, num_chunks):
    lines = s.splitlines()
    num_lines = len(lines)
    chunk_size = num_lines // num_chunks
    chunks = [lines[i:i+chunk_size] for i in range(0, num_lines, chunk_size)]
    # If there are leftover lines, add them to the last chunk
    if num_lines % num_chunks != 0:
        chunks[-1] += lines[num_chunks*chunk_size:]
    return ['\n'.join(chunk) for chunk in chunks]

In [8]:
output_csv = ""

# for text in split_string(table_text, 5):
for (i,text) in enumerate(tqdm.tqdm(split_string(table_text, 12))):

    prompt = f"""
Here is a table of hardware that is used for mining bitcoin:

{hardware}




I have analyzed posts from a bitcoin forum and created this csv of hardware:

{text.replace("Bitmain ","").replace("  "," ")}



Using the hardware table, rewrite this csv with columns: row_index,hardware_name,hardware_index
If a hardware name is unclear or not in the table of mining hardware, write "unknown" for the hardware_index value. The names don't need to match exactly, use common sense. For example, "Sapphire 7950 Vapor X" refers to the 7950 in the hardware table.

    """.strip()



    # print(prompt)
    # print("\n"*100)

    response = get_openai_response(prompt) # the response is a csv as a string with columns row_index,hardware_name,hardware_index

    print(response)
    print("\n"*100)

    if i == 0:
        output_csv = response
    else:
        output_csv += "\n" + response.replace("row_index,hardware_name,hardware_index", "").strip()


print(output_csv)
# save the output_csv
with open('datasets/extracted/name_index.csv', 'w') as file:
    file.write(output_csv)




  8%|▊         | 1/13 [01:06<13:20, 66.68s/it]

row_index,hardware_name,hardware_index
0,5870,118
1,6870,127
2,poclbm,unknown
3,HD5970,119
4,5870,118
5,5850,117
6,5870,118
7,Intel Graphics Media Accelerator GMA 4500M,unknown
8,5970,119
9,5870,118
10,Radeon 5970,119
11,Amazon EC2 Cluster GPU Instances,unknown
12,Nvidia 295,unknown
13,Thermaltake W0133RU ToughPower 1200W Modular Power Supply,unknown
14,iPhone,unknown
15,5870 bios,118
16,3x5850,117
17,ToughPower 1350W supply,unknown
18,non-reference Sapphire 5870,118
19,graphics cards,unknown
20,reference 5850,117
21,Intel E6300,unknown
22,Intel D975XBX,unknown
23,Coolermaster 700W Silent Pro,unknown
24,motherboards,unknown
25,2GB DDR2 667,unknown
26,PSU,unknown
27,Zotac ION,244
28,550w,unknown
29,69xx,unknown
30,5970,119
31,6990,131
32,5870,118
33,5850,117
34,5850,117
35,58xx,unknown
36,5830,116
37,2.5 dual core,unknown
38,MSI,unknown
39,5800 series cards,unknown
40,GeForce 9600 GSO,188
41,ATI 5xxx GPUs,unknown
42,Radeon 6990,131
43,5870s,118
44,5870s,118
45,HD5830,116
46,HD5830,116
4

 15%|█▌        | 2/13 [02:05<11:22, 62.09s/it]

row_index,hardware_name,hardware_index
100,Rig,unknown
101,HD 5970s,119
102,x6500,92
103,x6500 fpga,92
104,BeagleBone,unknown
105,BitForce,84
106,ATI 3800,unknown
107,ATI 3870x2,unknown
108,BTCMiner,unknown
109,bemicro,unknown
110,ZTEX boards,93
111,2 6850's,126
112,5830,116
113,BitForce Jalapeno,84
114,5830,116
115,5770,115
116,5850,117
117,Radeon 5970,119
118,cgminer,unknown
119,BFL Single,32
120,mod miner quad,90
121,FPGAs,unknown
122,5870m,unknown
123,Alienware M17xr2 laptop,unknown
124,DD-WRT router,unknown
125,Ztex boards,93
126,single sc,32
127,5830,116
128,6950,129
129,pentium 4 pc,unknown
130,6970,130
131,5850,117
132,Avlon,unknown
133,Avalon,20
134,BFL,32
135,Icarus FPGA,87
136,BFL Announcement Trolling Museum #2,unknown
137,Avalon,20
138,BFL,32
139,5970,119
140,Gigabyte GV-R797OC-3GD,unknown
141,Avalon ASIC,20
142,AVALON,20
143,Avalon,20
144,Avalon,20
145,BFGMiner,unknown
146,Avalon ASIC Batch 3,20
147,6950,129
148,6870,127
149,BFL ASIC,32
150,BFL 5 gh/s units,32
151,25 GH/s

 23%|██▎       | 3/13 [03:09<10:28, 62.84s/it]

...

row_index,hardware_name,hardware_index
1009,inline ethernet surge protector,unknown
1010,server PSUs,unknown
1011,fiber link,unknown
1012,surge arrestor,unknown
1013,APW3,unknown
1014,ATX psu,unknown
1015,Avalon 6,22
1016,Antminer S9,8
1017,HP Server Power Supplies,unknown
1018,Antminer S9,8
1019,BTMine v1 Miner,unknown
1020,Antminer s2,1
1021,Antminer s1,0
1022,Halong Mining DragonMint T1,unknown
1023,Antminer S9,8
1024,Avalon 741,24
1025,Antminer S9,8
1026,Innosilicon T2,unknown
1027,S9,8
1028,BFGMiner,unknown
1029,CGMiner,unknown
1030,CGWatcher 1.4.0,unknown
1031,Antminer S9,8
1032,Antminer S7,7
1033,$5600.00,unknown
1034,S9 Rackmount Self,unknown
1035,Antminer T9,unknown
1036,Antminer S15,unknown
1037,Antminer S9,8
1038,Whatsminer M20S,unknown
1039,S17,unknown
1040,T2,unknown
1041,T30,unknown
1042,T15,unknown
1043,S9k,unknown
1044,S15,unknown
1045,Delta QFR1212GHE,unknown
1046,T9+,unknown
1047,Beagleboard,unknown
1048,Noctua industrialPPC,unknown
1049,Avalon841,unknown
1050,Antmine

 92%|█████████▏| 12/13 [13:32<01:10, 70.85s/it]

row_index,hardware_name,hardware_index
1110,s9,8
1111,Avalon 1047,unknown
1112,Black arrow Prospero X1,unknown
1113,Asic S19j Pro,unknown
1114,Antminer S9,8
1115,S9,8
1116,S17 Pro,unknown
1117,Whatsminers,unknown
1118,L3,unknown
1119,Antminer S19,unknown
1120,S17,unknown
1121,S19J Pro 104T,unknown
1122,Avalon A1166 81T,unknown
1123,S17+,unknown
1124,S17,unknown
1125,Canaan Avalon 1246,unknown
1126,Antminer s9,8
1127,cpuminer,unknown
1128,M20s,unknown
1129,A1246,unknown
1130,S17,unknown
1131,Antminer,unknown
1132,Butterfly Labs,unknown
1133,S9,8
1134,S9k,unknown
1135,Compac F,unknown
1136,Apollo BTC,unknown
1137,T17,unknown
1138,Whatsminer,unknown
1139,Ebang Ebit 10.1,unknown
1140,T17,unknown
1141,S19 XP,unknown
1142,T19 Hydro,unknown
1143,741,24
1144,Apollo BTC,unknown
1145,A1066,unknown
1146,A1246 Series,unknown
1147,A1166,unknown
1148,A721,23
1149,821,26
1150,1047,unknown
1151,A10 Series,unknown
1152,A1246 93TH,unknown
1153,1041,unknown
1154,921,unknown
1155,A721,23
1156,841,unknown


100%|██████████| 13/13 [13:41<00:00, 63.19s/it]

row_index,hardware_name,hardware_index
1211,Antminer S15,unknown
1212,Goldshell KD2,unknown
1213,bitaxeUltra,unknown
1214,insilicon A11 pro,unknown
1215,Microbt,unknown
1216,Antminer S21,unknown
1211,Antminer S15,unknown
1212,Goldshell KD2,unknown
1213,bitaxeUltra,unknown
1214,insilicon A11 pro,unknown
1215,Microbt,unknown
1216,Antminer S21,unknown





































































































row_index,hardware_name,hardware_index
0,5870,118
1,6870,127
2,poclbm,unknown
3,HD5970,119
4,5870,118
5,5850,117
6,5870,118
7,Intel Graphics Media Accelerator GMA 4500M,unknown
8,5970,119
9,5870,118
10,Radeon 5970,119
11,Amazon EC2 Cluster GPU Instances,unknown
12,Nvidia 295,unknown
13,Thermaltake W0133RU ToughPower 1200W Modular Power Supply,unknown
14,iPhone,unknown
15,5870 bios,118
16,3x5850,117
17,ToughPower 1350W supply,unknown
18,non-reference Sapphire 5870,118
19,graphics cards,unknown
20,reference 5850,117
21,Intel E6300,unknown
22,In

# join the tables

In [37]:
name_index = pd.read_csv('datasets/extracted/name_index.csv') # columns: row_index,hardware_name,hardware_index
hardware_full = pd.read_csv('hardware_full.csv') # columns: hardware_name,hashrate,efficiency,hardware_index
price_only_cleaned = pd.read_csv('datasets/extracted/price_only_cleaned.csv') # columns: row_index,hardware_price
date_only = pd.read_csv('datasets/extracted/date_only.csv') # columns: row_index,date

# left join name_index and price_only_cleaned on row_index
df = pd.merge(name_index, price_only_cleaned, on='row_index', how='left')

# left join df and date_only on row_index
df = pd.merge(df, date_only, on='row_index', how='left')

# replace "unknown" with -1
df["hardware_index"]  = df["hardware_index"].replace("unknown", -1)

# left join df and hardware_full on hardware_index
df["hardware_index"] = df["hardware_index"].astype(int)
hardware_full["hardware_index"] = hardware_full["hardware_index"].astype(int)
hardware_full = hardware_full.drop(columns=['hardware_name'])
df = pd.merge(df, hardware_full, on='hardware_index', how='left')

# save it
df.to_csv('datasets/extracted/full_table.csv', index=False)

In [38]:
df.sample(20)

,row_index,hardware_name,hardware_index,hardware_price,date,hashrate,efficiency
951,951,Antminers,-1,NaN,2017-05-23,NaN,NaN
371,371,BLACK ARROW Bullet Run,-1,123$,2013-10-28,NaN,NaN
744,744,SP20,-1,NaN,2015-02-13,NaN,NaN
558,558,hex miner 15 Gigahashes USB Stick,-1,NaN,2014-03-12,NaN,NaN
143,143,Avalon,20,NaN,2013-03-26,300000,unknown
725,725,KNC Neptune,55,0.6BTC,2014-11-20,300000,1429
635,635,Corsair CX750M,-1,129.99$,2014-06-04,NaN,NaN
475,475,P2Pool,-1,NaN,2014-01-15,NaN,NaN
721,721,Spondoolies Tech SP20,70,NaN,2014-11-10,170000,1545
871,871,r7 370,-1,NaN,2016-03-01,NaN,NaN


# fix btc prices

In [39]:
df = pd.read_csv('datasets/extracted/full_table.csv')
df['hardware_price'] = df['hardware_price'].astype(str)

In [40]:
from torch import save, load
date_to_btc_price = load('date_open_dict.pt')

In [41]:
# for every row where hardware_price contains "BTC", get the price in dollars using the dict and the amount of BTC
for index, row in df.iterrows():
    if "BTC" in row['hardware_price']:
        btc_price = row['hardware_price'].replace(",", "").split("BTC")[0].strip()
        try:
            btc_price = float(btc_price)
        except:
            btc_price = -1
        date = row['date']
        
        # get the price in dollars
        dollars_price = date_to_btc_price[date] * btc_price
        dollars_price = round(dollars_price, 3)

        # replace the price in the dataframe
        if not btc_price == -1:
            df.at[index, 'hardware_price'] = f"{dollars_price}$"
        else:
            df.at[index, 'hardware_price'] = f""

# drop rows where hardware_price contains €
df = df[df['hardware_price'].str.contains("€") == False]

#remove all characters except for numbers and .
df['hardware_price'] = df['hardware_price'].apply(lambda x: re.sub(r"[^0-9.]", "", x).strip())

#columns are date,hardware_name,hardware_price,hashrate,efficiency
#they should be date,hardware_name,hardware_price_usd,Mhash/s/$,Mhash/J
#rename columns
df = df.rename(columns={"hardware_price": "hardware_price_usd", "hashrate": "Mhash/s/$", "efficiency": "Mhash/J"})

# save the dataframe
df.to_csv('datasets/extracted/final.csv', index=False)
        